# Check and setup

In [ ]:
pip install keras-self-attention
pip install scikit-optimize

import os
from collections import Counter
from random import choice
import time
from urllib.request import Request, urlopen

import numpy as np
import pandas as pd


from IPython.display import Image

import keras
from keras_self_attention import SeqSelfAttention

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import models

from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.utils import use_named_args

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Tensorflow version checking
print("Tensorflow version " + tf.__version__)
print(pd.__version__)

Tensorflow version 2.12.0
1.5.3


In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:
# define some parameters

## size of the train set
train_percent_split = 0.9

# first hyperparameters of the RNN model
epochs = 1
dropout = 0
embedding_size = 128
hidden_size_lstm = 128
learning_rate = 0.00276
attention_width = 20
add_dense_layer = True
hidden_size_dense = 128

validation_split = 0.1
n_reco = 20
n_min_interactions = 20
max_length_seq = 20
nb_last_item = 20
max_users = 5000

# iterations number for optimization with scikit-optimize lib
n_random_starts = 2
n_calls = 15

# Data importation

In [ ]:
# paths definition
path = '/content/drive/MyDrive/Colab/'
book_fp = os.path.join(path, 'BX-Books.csv')
user_fp = os.path.join(path, 'BX-Users.csv')
rating_fp = os.path.join(path, 'BX-Book-Ratings.csv')

In [ ]:
# files reading
def read_file(fp):
    data = pd.read_csv(fp,
                       sep=';',
                       encoding='latin-1',
                       low_memory=False,
                       header=0,
                       error_bad_lines= False)
    return data
    
item_df = read_file(book_fp)
# user_df = read_file(user_fp)
# rating_df = read_file(rating_fp)


<ipython-input-39-011433955c77>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(fp,
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9
Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9
Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9
Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
Skip

In [ ]:
print(user_df.shape)
print(user_df.head())



(278858, 3)
   user                            Location   Age
0     1                  nyc, new york, usa   NaN
1     2           stockton, california, usa  18.0
2     3     moscow, yukon territory, russia   NaN
3     4           porto, v.n.gaia, portugal  17.0
4     5  farnborough, hants, united kingdom   NaN


In [ ]:
print(item_df.shape)
print(item_df.head())

(271360, 8)
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images

In [ ]:
print(rating_df.shape)
print(rating_df.head())

(1149780, 3)
     user           item  rating
0  215225     3404145054       0
1  214157  9782878110753       6
2  186564     8483108445       5
3  113519     0345343581       0
4  166123     0449221490       0


- Sorted by Used-ID

In [ ]:
rating_df = rating_df.sample(frac=1).reset_index(drop=True)

- check NA value on rating_df

In [ ]:
rating_df.isna().sum()

user      0
item      0
rating    0
dtype: int64

# Data preparation 

- rename columns

In [ ]:
colname_mapping = {
    'ISBN': 'item',
    'User-ID': 'user',
    'Book-Rating': 'rating',
    'Book-Title': 'name',
    'Book-Author': 'author',
    'Image-URL-M': 'image',
    'Publisher': 'publisher',
    'Year-Of-Publication': 'year'
}

# rating_df = rating_df.rename(columns=colname_mapping)
item_df = item_df.rename(columns=colname_mapping)
# user_df = user_df.rename(columns=colname_mapping)

In [ ]:
items = list(set(item_df.item.unique().tolist() + rating_df.item.unique().tolist()))
users = user_df.user.unique() 
print(f'number of unique items: {len(items)}\nnumber of unique users: {len(users)}')

number of unique items: 341765
number of unique users: 278858


In [ ]:
rating_df = rating_df[rating_df.item.isin(items)]
item_df = item_df[item_df.item.isin(items)]
rating_df.shape, item_df.shape

((1149780, 3), (271360, 8))

In [ ]:
user_to_token = {user: int(token) for token, user in enumerate(users)}
token_to_item = {token: user for user, token in user_to_token.items()}
item_to_token = {item: int(token) for token, item in enumerate(items)}
token_to_item = {token: item for item, token in item_to_token.items()}

rating_df['user_id'] = rating_df['user'].map(user_to_token)
rating_df['item_id'] = rating_df['item'].map(item_to_token).dropna().astype(int)

item_df['item_id'] = item_df['item'].map(item_to_token)
user_df['user_id'] = user_df['user'].map(user_to_token)

In [ ]:
item_df.item_id.nunique(), rating_df.item_id.nunique()

(271360, 340556)

In [ ]:
item_df.head()

,item,name,author,year,publisher,Image-URL-S,image,Image-URL-L,item_id
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,318864
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11626
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,62326
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,17816
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,238075


- In4 book

In [ ]:
def print_single_item_characteristics(item_id=None, item_df=item_df):
    if not item_id:
        item_id = choice(item_df.item_id)
    if item_id not in set(item_df.item_id):
        print(f'item_id {item_id} not in df')
        return None

    item_df = item_df[item_df['item_id'] == item_id]
    url = item_df.image.values[0]
    response =  Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    print(f'item_id: {item_id}; '
          f'name: {item_df.name.values[0]}; '
          f'author: {item_df.author.values[0]} '
          f'publisher: {item_df.publisher.values[0]} '
          f'year: {item_df.year.values[0]}')

def print_items_characteristics(item_id_list):
    for item_id in item_id_list:
        display(print_single_item_characteristics(item_id=item_id))

In [ ]:
n_rating_by_user = rating_df.user_id.value_counts()

In [ ]:
n_rating_by_user.describe()

count    105283.000000
mean         10.920851
std          90.562825
min           1.000000
25%           1.000000
50%           1.000000
75%           4.000000
max       13602.000000
Name: user_id, dtype: float64

- check some popular books

In [ ]:
top3_item_id = list(rating_df.item_id.value_counts().index.values[:3])
top3_item_id

[287409, 274473, 293823]

In [ ]:
print_items_characteristics(item_id_list=top3_item_id)

item_id: 287409; name: Wild Animus; author: Rich Shapero publisher: Too Far year: 2004


None

item_id: 274473; name: The Lovely Bones: A Novel; author: Alice Sebold publisher: Little, Brown year: 2002


None

item_id: 293823; name: The Da Vinci Code; author: Dan Brown publisher: Doubleday year: 2003


None

- interactions per user

In [ ]:
user_occurence = Counter(rating_df.user).most_common()
print(user_occurence[:20])

[(11676, 13602), (198711, 7550), (153662, 6109), (98391, 5891), (35859, 5850), (212898, 4785), (278418, 4533), (76352, 3367), (110973, 3100), (235105, 3067), (230522, 2991), (16795, 2948), (234623, 2674), (36836, 2529), (52584, 2512), (245963, 2507), (204864, 2504), (55492, 2459), (185233, 2448), (171118, 2421)]


- user > n interations

In [ ]:
def user_with_n_interaction(data, n):
    print(f'length before filtering: {len(data)}.')
    user_occurence = Counter(data.user)

    user_to_keep = [
        user
        for user, occ in user_occurence.items()
        if occ >= n
    ]

    data_filtered = data[data['user'].isin(user_to_keep)]
    print(f'length after filtering: {len(data_filtered)}.')
    return data_filtered

rating_df = user_with_n_interaction(data=rating_df, n=n_min_interactions)

length before filtering: 1149780.
length after filtering: 888884.


In [ ]:
split_ind = int(len(rating_df) * train_percent_split)
train, test = rating_df[:split_ind], rating_df[split_ind:]
print(f'shape of train: {train.shape}\nshape of test: {test.shape}')

shape of train: (799995, 5)
shape of test: (88889, 5)


# Sequences preparation for the RNN model

In [ ]:
#function to prepare sequence data
def prepare_sequences(data, users, item_to_token, max_length=20, 
                      one_hot_encoding=False):
    
    print('preparing sequences')
    
    #generate sequences - see https://stackoverflow.com/questions/36864699/pandas-pivot-dataframe-with-unequal-columns        
    data = pd.concat([
        pd.DataFrame(
            {
                g:[0] * (max_length+1-len(d['item_id'].tolist()[-max_length-1:])) + d['item_id'].tolist()[-max_length-1:]
            }
        )
        for g,d in data.groupby('user_id')], axis=1)
 
    
    #from pandas dataframe to numpy array
    data = data.transpose().values
        
    #transpose and build the arrays
    x = np.array([i[:-1] for i in data])
    y = np.array([i[1:] for i in data])
            
    #build the one-hot encoding, if we want
    if one_hot_encoding:
        y = np_utils.to_categorical(y, len(item_to_token)+1)
    else:
        y = np.expand_dims(y, -1)
    
    print('sequences prepared')
        
    return (x, y)

#function to extract prediction from keras model at last timestep
def predict_last_timestep(model, data):
    #calculate the model output
    prediction = model.predict(data)
    #keep only the prediction at the final timestep
    return prediction[-1]

In [ ]:
x, y = prepare_sequences(data=train, users=users, item_to_token=item_to_token, max_length=max_length_seq)

preparing sequences
sequences prepared


# Keras model definition

In [ ]:
def keras_model(hidden_size_lstm=hidden_size_lstm, 
                learning_rate=learning_rate, 
                dropout=dropout,
                attention_width=attention_width,
                embedding_size=embedding_size,
                add_dense_layer=add_dense_layer,
                hidden_size_dense=hidden_size_dense,
                embedding_matrix=None,
                item_to_token=item_to_token):
    
    with strategy.scope():
        embedding_layer = Embedding(len(item_to_token)+1,
                                    embedding_size,
                                    weights=embedding_matrix,
                                    mask_zero=True)

        model = Sequential()
        model.add(embedding_layer)
        model.add(LSTM(units=hidden_size_lstm,
                       activation='tanh', dropout=dropout,
                       return_sequences=True))
        model.add(SeqSelfAttention(attention_activation='sigmoid',
                                   attention_width=attention_width,
                                   history_only=True))
        if add_dense_layer:
            model.add(Dense(units=hidden_size_dense, activation='relu'))
        model.add(Dense(units=len(item_to_token)+1, activation='softmax'))
        optimizer = Adam(lr=learning_rate)
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
                  metrics=['sparse_categorical_accuracy'])

    return model

# Model training 

In [ ]:
#prepare the model
model = keras_model(hidden_size_lstm=hidden_size_lstm, 
                    learning_rate=learning_rate, 
                    dropout=dropout, 
                    embedding_size=embedding_size)  

history = model.fit(x, y, 
                    epochs=epochs,
                    validation_split=validation_split,
                    batch_size=64,
                    verbose=1)

print(history.history['val_sparse_categorical_accuracy'][-1])


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


104/104 [==============================] - 2508s 24s/step - loss: 12.3798 - sparse_categorical_accuracy: 0.0015 - val_loss: 12.1052 - val_sparse_categorical_accuracy: 0.0016
0.001584240235388279


# Recommendations prediction

In [ ]:
common_user = list(set(train.user_id).intersection(test.user_id))
common_user[:10]

[32772, 262150, 163850, 196620, 98321, 32792, 131098, 229408, 32801, 163874]

In [ ]:
print(f'There are {len(common_user)} common users between the train and test set')

There are 7175 common users between the train and test set


In [ ]:
def reco_from_item_id_interacted(item_id_interacted, model=model, n_reco=10):
    predictions = predict_last_timestep(model=model,
                                        data=[item_id_interacted]).argsort()[0][:n_reco]
    return list(predictions)

def build_user_to_interacted(user_id, train=train):
    return train[train['user_id'] == user_id]['item_id'].dropna().unique().tolist()

#item_id_interacted = build_user_to_interacted(user_id=choice(user_train))
#len(item_id_interacted)

In [ ]:
def user_to_last_visited_item_id_dict(train: pd.DataFrame,
                                      user_list: list,
                                      nb_last_item: int=None) -> dict:
    """
    Return a dictionary mapping user to last visited items id.
    input:
            :train: pd.DataFrame, training set
            :user_list: list, head_visitor_id
            :nb_last_item: int, number of last interacted items
                           to use to predict recommendation
    output:
            :: dict, mapping the user (head_visitor_id) to the last
               visited items
    """
    if nb_last_item:
        return train.groupby('user_id')['item_id'].apply(lambda g: g.values
                                                      .tolist()[-nb_last_item:]).to_dict()
    else:
        return train.groupby('user_id')['item_id'].apply(lambda g: g.values
                                                      .tolist()).to_dict()

user_to_last_visited_item_id_dict = user_to_last_visited_item_id_dict(train=train,
                                                                      user_list=train.user_id.unique().tolist(),
                                                                      nb_last_item=nb_last_item)

In [ ]:
list(user_to_last_visited_item_id_dict.items())[:5]

[(182,
  [185991,
   124005,
   283226,
   251666,
   71978,
   81843,
   261374,
   299757,
   157939,
   92203,
   275836,
   15629,
   13882,
   164662,
   317077,
   288803,
   192643,
   40547,
   101366,
   56001]),
 (241,
  [4026,
   105542,
   4594,
   96169,
   190948,
   128711,
   330000,
   39120,
   292740,
   226178,
   65388,
   224980,
   19005,
   19384,
   256727,
   279696,
   182050,
   187295,
   241176,
   304654]),
 (242,
  [189953,
   146477,
   131964,
   67881,
   174827,
   303718,
   274473,
   184832,
   165559,
   37633,
   262910,
   149098,
   19073,
   53613,
   47281,
   44497,
   278043,
   116381,
   2425,
   330816]),
 (253,
  [86240,
   329822,
   336229,
   22239,
   190183,
   50307,
   211140,
   50948,
   229516,
   159063,
   170832,
   120501,
   134709,
   102825,
   237986,
   293703,
   296179,
   328257,
   19898,
   221293]),
 (382,
  [309225,
   205832,
   225550,
   251893,
   60018,
   150816,
   215505,
   230219,
   6550,
   20041,


In [ ]:
def predict_for_one(
    model,
    user_id,
    n_reco,
    user_to_last_visited_item_id_dict=user_to_last_visited_item_id_dict
):
    item_id_interacted = user_to_last_visited_item_id_dict[user_id]
    reco_id = reco_from_item_id_interacted(item_id_interacted,
                                           model=model,
                                           n_reco=n_reco)
    return reco_id

In [ ]:
def predict(
    model,
    user_id_list,
    n_reco,
    user_to_last_visited_item_id_dict=user_to_last_visited_item_id_dict,
    max_users=None, # reduce computation time
):
    start = time.time()
    print(f'recommendation computation for {len(user_id_list[:max_users])} users.')
    print(f'n_reco={n_reco}')
    reco_dict = {
        user_id: predict_for_one(
            model=model,
            user_id=user_id,
            n_reco=n_reco,
            user_to_last_visited_item_id_dict=user_to_last_visited_item_id_dict
        )
        for user_id in user_id_list[:max_users]
    }
    print(f'predict for {len(user_id_list[:max_users])} spent {round(time.time()-start, 2)} s.')
    return reco_dict
#max_users=100
#n_reco=50
reco_dict = predict(
    model=model,
    user_id_list=common_user,
    user_to_last_visited_item_id_dict=user_to_last_visited_item_id_dict,
    max_users=max_users,
    n_reco=n_reco)

recommendation computation for 5000 users.
n_reco=20
1/1 [==============================] - 0s 156ms/step


In [ ]:
list(reco_dict.items())[:3]

# Performance metrics computation

In [ ]:
#Function to calculate, precision, recall and coverage
def statistics_at_k(reco_dict,
                    test_df,
                    train_df,
                    calculate_precision=True,
                    calculate_recall=True, 
                    calculate_coverage=True): 
    '''
    reco_dict: dictionary with the uid as key, list of items recommended as attribute
    test_df: dataframe of user-item interactions
    '''     
    #calculate precision
    if calculate_precision:
        k_relevant = 0
        k_total = 0
        for uid, iid in reco_dict.items():
            iid_test = set(test_df[test_df['user_id'] == uid]['item_id'])
            for j in iid:
                k_total += 1
                if j in iid_test:
                    k_relevant += 1
        if not k_total:
            precision = 0
        else:
            precision = k_relevant/k_total
        print(f'precision={precision}')    
    else:
        precision = None
        
    #calculate precision
    if calculate_recall:
        k_relevant = 0
        k_total = 0
        for uid, iid in reco_dict.items():
            for j in list(test_df[test_df['user_id'] == uid]['item_id']):
                k_total += 1
                if j in set(iid):
                    k_relevant += 1
        
        if not k_total:
            recall = 0
        else:
            recall = k_relevant/k_total
        print(f'recall={recall}')
    else:
        recall = None
        
    #calculate coverage
    if calculate_coverage:
        nb_recommended = len(set(sum(reco_dict.values(), [])))
        nb_total = len(train_df['item_id'].unique())
        coverage = nb_recommended/nb_total
        print(f'coverage={coverage}')
    else:
        coverage = None
    
    return precision, recall, coverage

In [ ]:
# statistics with RNN recommendation
statistics_at_k(reco_dict=reco_dict,
                test_df=test,
                train_df=train,
                calculate_precision=True,
                calculate_recall=True, 
                calculate_coverage=True)

In [ ]:
topn = list(train.item_id.value_counts().index.values[:n_reco])
topn

In [ ]:
reco_topn_dict = {
    user_id: topn
    for user_id in reco_dict.keys() 
}

In [ ]:
list(reco_topn_dict.items())[:3]

In [ ]:
# statistics with top n recommendation
statistics_at_k(reco_dict=reco_topn_dict,
                test_df=test,
                train_df=train,
                calculate_precision=True,
                calculate_recall=True, 
                calculate_coverage=True)

# Visualization ex

In [ ]:
item_id_interacted = [197383]
item_id_pred = predict_last_timestep(
    model=model,data=[item_id_interacted]
).argsort()[0][:20]

In [ ]:
# past interactions
item_df[item_df['item_id'].isin(item_id_interacted)]

In [ ]:
# visualization of the pas
print_items_characteristics(item_id_list=item_id_interacted)

In [ ]:
# predictions
item_df[item_df['item_id'].isin(item_id_pred)]

In [ ]:
item_id_pred

In [ ]:
print_items_characteristics(item_id_list=item_id_pred)

In [ ]:
del history
del model

# RNN model hyperparameter optimization

In [ ]:
# define the dimension to search
dim_epochs = Integer(low=1, high=15, name='epochs')
dim_hidden_size_lstm = Categorical(categories=[32, 64, 128, 256], name='hidden_size_lstm')
dim_learning_rate = Real(low=1e-4, high=5e-1, prior='log-uniform',
                         name='learning_rate')
dim_attention_width = Integer(low=1, high=50, name='attention_width')
dim_dropout = Real(low=0, high=0.9, name='dropout')
dim_embedding_size = Categorical(categories=[64, 128, 256, 512], name='embedding_size')
dim_add_dense_layer = Categorical(categories=[True, False], name='add_dense_layer')
dim_hidden_size_dense = Categorical(categories=[32, 64, 128, 256], name='hidden_size_dense')

dimensions = [dim_epochs,
              dim_hidden_size_lstm,
              dim_learning_rate,
              dim_attention_width,
              dim_dropout,
              dim_embedding_size,
              dim_add_dense_layer,
              dim_hidden_size_dense]

epochs = 3
dropout = 0
embedding_size = 256
hidden_size_lstm = 64
learning_rate = 0.00276

In [ ]:
def optimize(dimensions=dimensions, n_calls=15, n_random_starts=3, verbose=1, x0=None):
    print(dimensions)
    @use_named_args(dimensions=dimensions)
    def fitness(**params):
        print(f'params={params}')
        model = keras_model(hidden_size_lstm=params['hidden_size_lstm'],
                            learning_rate=params['learning_rate'],
                            attention_width=params['attention_width'],
                            dropout=params['dropout'],
                            embedding_size=params['embedding_size'],
                            add_dense_layer=params['add_dense_layer'],
                            hidden_size_dense=params['hidden_size_dense'])  

        history = model.fit(x, y,
                            epochs=params['epochs'],
                            validation_split=validation_split,
                            batch_size=64,
                            verbose=verbose)
        sca = history.history['val_sparse_categorical_accuracy'][-1]
        print(f'##sca={sca}## with params={params}')
        del history
        del model
        return -1.0 * sca
    
    res = gp_minimize(func=fitness,
                      dimensions=dimensions,
                      acq_func='EI', # Expected Improvement.
                      n_calls=n_calls,
                      n_random_starts=n_random_starts,
                      x0=x0)
    print(f'best accuracy={-1.0 * res.fun} with {res.x}')
    return res

In [ ]:
res = optimize(dimensions=dimensions,
               n_calls=n_calls,
               n_random_starts=n_random_starts,
               x0=[3, 256, 0.001, 40, 0, 512, True, 128])